In [0]:
# Class IT-533 Week #3  - Wrangling Data: Preprocessing and Visualizing
# Summer 2020
# Lab #2 - UCI_Credit_Card.csv
# By Wei Geng(David) and Anusha Bale
# Created on: 05/31/2020
# Honor Code: “I have neither given or received, nor have I tolerated others' use of unauthorized aid.”

In [0]:
# install pandas in case you dont have pandas pre-installed
%pip install pandas

In [3]:
# Convert to pandas DataFrame
import io
import numpy as np
import pandas as pd
remen_df = pd.read_csv('https://raw.githubusercontent.com/intelspeedstep/AMOD533/master/ramen-ratings.csv')
remen_df.head()

,Review #,Brand,Variety,Style,Country,Stars,Top Ten
0,2580,New Touch,T's Restaurant Tantanmen,Cup,Japan,3.75,NaN
1,2579,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,1,NaN
2,2578,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,2.25,NaN
3,2577,Wei Lih,GGE Ramen Snack Tomato Flavor,Pack,Taiwan,2.75,NaN
4,2576,Ching's Secret,Singapore Curry,Pack,India,3.75,NaN


In [4]:
# output the data attributes and its datatypes, plus the shape of the dataset
print(remen_df.info(verbose=True))
print(remen_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Review #  2580 non-null   int64 
 1   Brand     2580 non-null   object
 2   Variety   2580 non-null   object
 3   Style     2578 non-null   object
 4   Country   2580 non-null   object
 5   Stars     2580 non-null   object
 6   Top Ten   41 non-null     object
dtypes: int64(1), object(6)
memory usage: 141.2+ KB
None
(2580, 7)


### <font color = 'orange'>Question #1: Data Transformation: Pick three variables whose data types may be more useful as another data type and use R or Python to transform them to numeric or character/ categorical format.  Then, explain in no less than 3 sentences what this transformation will allow you to accomplish analytically later on.</font>
<br> <font color = 'green'>Answer:1) Transforming "Review #" from numeric to categorical would avoid of treating the ID variable as numbers for calculations. 2) Transforming "stars" from categorical to numeric would allow me to slice and dice the data by comparing star rating to see if there is disparities among brand, country, style and top 10 ranked restaurants 3) Transforming "Top Ten" from categorical variable to binary would make comparison by 'Top ten ranked' vs 'No top ten ranked'. This would be easier for analytics.
</font></br>

In [5]:
# Exam the levels of each variable which I picked --  sex, education and marriage
print(remen_df['Review #'].value_counts(normalize=True,dropna=False))
print(remen_df['Stars'].value_counts(normalize=True,dropna=False))
print(remen_df['Top Ten'].value_counts(normalize=True,dropna=False))

2047    0.000388
1240    0.000388
1196    0.000388
1194    0.000388
1192    0.000388
          ...   
2478    0.000388
429     0.000388
2476    0.000388
427     0.000388
2049    0.000388
Name: Review #, Length: 2580, dtype: float64
4          0.148837
5          0.143023
3.75       0.135659
3.5        0.126357
3          0.067054
3.25       0.065891
4.25       0.055426
4.5        0.051163
2.75       0.032946
2          0.026357
2.5        0.025969
4.75       0.024806
1.5        0.014341
1.75       0.010465
1          0.010078
0          0.010078
2.25       0.008140
0.5        0.005426
0.25       0.004264
1.25       0.003876
5.0        0.003876
3.50       0.003488
5.00       0.002713
4.00       0.002326
4.3        0.001550
4.0        0.001163
Unrated    0.001163
4.50       0.001163
3.8        0.001163
3.1        0.000775
2.9        0.000775
2.8        0.000775
4.125      0.000775
2.3        0.000775
3.0        0.000775
1.1        0.000775
2.1        0.000388
0.75       0.000388
3.3     

In [6]:
# I would pick Review #, Stars and Top Ten to transform them from numeric to categorical(object)
remen_df['Review #']=remen_df['Review #'].astype(object)
remen_df.loc[remen_df["Stars"]=='Unrated','Stars']=np.nan
remen_df['Stars']=remen_df['Stars'].astype(float)
remen_df.loc[remen_df['Top Ten'].notnull(),'Top Ten Indicator']=1
remen_df[['Review #','Stars','Top Ten Indicator']].info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Review #           2580 non-null   object 
 1   Stars              2577 non-null   float64
 2   Top Ten Indicator  41 non-null     float64
dtypes: float64(2), object(1)
memory usage: 60.6+ KB


### <font color = 'orange'>Question #2: Data Substitution: Use R or Python to substitute values in at least two variables. Explain in no less than 3 sentences how you decided what values to use for substitution and, in addition to your explanation, show your code.</font>
<br> <font color = 'green'>Answer: There are two variables I used substitute method to replace values. 1) We substitute missing/unrated value in "Stars" to 0. It is better to in this way when we avarage the star rating for restaurants. 2) It is better to replace the missings by giving 0 to the value, again it would be easier for avarage rate calculation.  </font>


In [7]:
remen_df.loc[remen_df['Stars']==np.nan,'Stars']=0
remen_df['Top Ten Indicator'].fillna(0,inplace=True)
print('Values in Stars Variable are: ',remen_df['Stars'].unique())
print('Values in Top Ten Indicator Variable are: ',remen_df['Top Ten Indicator'].unique())

Values in Stars Variable are:  [3.75  1.    2.25  2.75  4.75  4.    0.25  2.5   5.    4.25  4.5   3.5
   nan 1.5   3.25  2.    0.    3.    0.5   3.8   4.3   2.3   3.3   1.75
 0.75  1.25  1.1   2.1   0.9   3.1   4.125 3.125 2.125 2.9   0.1   2.8
 3.7   3.4   3.6   2.85  3.2   3.65  1.8  ]
Values in Top Ten Indicator Variable are:  [0. 1.]


### <font color = 'orange'>Question #3: Data Reduction: Pick two variables that could be simplified, either by combining two or more attributes (=columns) into one, or by binning or bucketing, then use R or Python to accomplish this.  Explain in no less than 3 sentences what benefit this may provide for any later analysis. </font>
<br> <font color = 'green'>Answer: I pick country and style to simplify, first. I would just concatenate two variables to one. I think country and style has one to many relations, so, using country and style one columns can be more efficient to present more information than having two separate columns
The second variable that I would like to simplify is Stars. I will use binning method to group values into different value ranges. In this way, the data can be show is a way for better understanding.

In [8]:
# Simplify two variables by combining country and style into to one varable 'country_and_style'
remen_df['country_and_style']=remen_df['Country']+'-'+remen_df['Style']
remen_df['country_and_style'].drop_duplicates().sort_values().reset_index(drop=True).dropna()

0          Australia-Cup
1         Australia-Pack
2        Bangladesh-Pack
3             Brazil-Cup
4            Brazil-Pack
             ...        
82    United States-Pack
83          Vietnam-Bowl
84           Vietnam-Cup
85          Vietnam-Pack
86          Vietnam-Tray
Name: country_and_style, Length: 87, dtype: object

In [9]:
# Simplify Stars variables by binning the star rating into 7 category values
remen_df['Stars']=pd.qcut(remen_df['Stars'],7,precision=2,duplicates='drop')
remen_df['Stars'].drop_duplicates().sort_values().reset_index(drop=True).dropna()

0    (-0.01, 2.75]
1     (2.75, 3.25]
2     (3.25, 3.75]
3      (3.75, 4.0]
4       (4.0, 5.0]
Name: Stars, dtype: category
Categories (5, interval[float64]): [(-0.01, 2.75] < (2.75, 3.25] < (3.25, 3.75] < (3.75, 4.0] <
                                    (4.0, 5.0]]

### <font color = 'orange'>Question #4: Now that you have a better dataset, list three (3) important organizational analytical questions that management could ask about the dataset in order to help them make business or organizational decisions.  </font>
<br> <font color = 'green'>Answer: Below are the question I think the management would ask: 1) Management team need to better understand the remen noodle market share, so could you breakdown the remen restaurant by country_style? 2) Which is the star rating that get most reviewed? 3) and Which is the country and style are more popular with higher ratings?   </font>